In [1]:
"""
absolute_quality_eval.py

Evaluate 3D model quality WITHOUT ground truth
Measures absolute quality indicators rather than relative comparison
"""

import numpy as np
import trimesh
from scipy.spatial import cKDTree
from PIL import Image, ImageDraw
import torch
import clip
import sys

sys.argv = [x for x in sys.argv if x != "-f"]


# =====================================
# 1. GEOMETRIC QUALITY
# =====================================

def surface_smoothness(mesh, k=10):
    """
    Measure surface smoothness via local normal variation
    Lower is smoother (range: 0-2, ideal < 0.3)
    """
    normals = mesh.vertex_normals
    tree = cKDTree(mesh.vertices)
    
    variations = []
    sample_size = min(1000, len(mesh.vertices))  # Sample for speed
    sample_indices = np.random.choice(len(mesh.vertices), sample_size, replace=False)
    
    for idx in sample_indices:
        _, neighbors = tree.query(mesh.vertices[idx], k=k+1)
        neighbor_normals = normals[neighbors[1:]]  # Exclude self
        
        # Cosine similarity between normal and neighbors
        cos_sim = np.dot(neighbor_normals, normals[idx])
        variation = 1 - np.mean(cos_sim)  # 0 = perfect alignment
        variations.append(variation)
    
    return float(np.mean(variations))


def triangle_quality(mesh):
    """
    Aspect ratio of triangles (width/height)
    Returns: mean, std, % of bad triangles
    Ideal: mean close to 1.0, few outliers
    """
    edges_per_face = mesh.edges_unique_length[mesh.faces_unique_edges]
    
    ratios = []
    for e in edges_per_face:
        e_sorted = np.sort(e)
        ratio = e_sorted[2] / (e_sorted[0] + 1e-10)  # Longest / shortest
        ratios.append(ratio)
    
    ratios = np.array(ratios)
    bad_ratio = np.sum(ratios > 5.0) / len(ratios)  # Threshold: 5:1
    
    return {
        'mean': float(np.mean(ratios)),
        'std': float(np.std(ratios)),
        'bad_triangle_ratio': float(bad_ratio)
    }


def resolution_score(mesh):
    """
    Vertex density and polygon count
    More vertices = higher detail (but not always better)
    """
    volume = mesh.volume if mesh.is_volume else mesh.bounding_box.volume
    surface_area = mesh.area
    
    return {
        'vertex_count': len(mesh.vertices),
        'face_count': len(mesh.faces),
        'vertices_per_unit_area': len(mesh.vertices) / (surface_area + 1e-10),
        'volume': float(volume),
        'surface_area': float(surface_area)
    }


# =====================================
# 2. TOPOLOGY QUALITY (from previous code)
# =====================================

def topology_quality_score(mesh):
    """
    Compute overall topology quality (0-1, higher is better)
    """
    scores = []
    weights = []
    
    # Watertight bonus
    if mesh.is_watertight:
        scores.append(1.0)
        weights.append(2.0)
    else:
        scores.append(0.0)
        weights.append(2.0)
    
    # Check manifold edges
    try:
        edge_faces = mesh.faces_sparse.tocsr()
        edges_per_face_count = np.array((edge_faces > 0).sum(axis=0)).flatten()
        manifold_ratio = np.sum(edges_per_face_count == 2) / len(edges_per_face_count)
        scores.append(manifold_ratio)
        weights.append(2.0)
    except:
        pass
    
    # Degenerate faces
    face_areas = mesh.area_faces
    non_degen_ratio = 1 - (np.sum(face_areas < 1e-10) / len(face_areas))
    scores.append(non_degen_ratio)
    weights.append(1.0)
    
    # Duplicate faces
    unique_faces = len(np.unique(np.sort(mesh.faces, axis=1), axis=0))
    non_dup_ratio = unique_faces / len(mesh.faces)
    scores.append(non_dup_ratio)
    weights.append(1.0)
    
    # Weighted average
    if len(scores) > 0:
        return float(np.average(scores, weights=weights))
    return 0.0


# =====================================
# 3. SEMANTIC COHERENCE (CLIP)
# =====================================

def semantic_coherence(mesh, text_prompt, device=None):
    """
    Check if model matches text description
    text_prompt: e.g., "a hair dryer", "a chair"
    Returns: 0-1 score (higher = better match)
    """
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"
    
    try:
        model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
        model.eval()
        
        # Render simple views
        imgs = render_views(mesh)
        combined = combine_images(imgs)
        
        # Encode image and text
        image = preprocess(combined).unsqueeze(0).to(device)
        text = clip.tokenize([text_prompt]).to(device)
        
        with torch.no_grad():
            image_features = model.encode_image(image).float()
            text_features = model.encode_text(text).float()
            
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            
            similarity = (image_features @ text_features.T).cpu().item()
        
        del model, image, text
        if device == "cuda":
            torch.cuda.empty_cache()
        
        return float(similarity)
    
    except Exception as e:
        print(f"Warning: Semantic coherence failed: {e}")
        return None


def render_views(mesh, image_size=224):
    """Simple orthographic rendering"""
    verts = mesh.vertices.copy()
    verts -= verts.mean(axis=0)
    scale = np.max(np.linalg.norm(verts, axis=1))
    if scale > 0:
        verts /= scale
    
    imgs = []
    for projection in [verts[:, [0, 1]], verts[:, [2, 1]], verts[:, [0, 2]]]:
        margin = int(0.05 * image_size)
        coords = ((projection + 1.0) * 0.5 * (image_size - 2*margin)) + margin
        coords = np.round(coords).astype(int)
        coords[:, 0] = np.clip(coords[:, 0], 0, image_size-1)
        coords[:, 1] = np.clip(coords[:, 1], 0, image_size-1)
        
        img = Image.new("RGB", (image_size, image_size), (255, 255, 255))
        draw = ImageDraw.Draw(img)
        for x, y in coords:
            draw.ellipse([x-1, image_size-y-1, x+1, image_size-y+1], fill=(30, 30, 30))
        imgs.append(img)
    
    return imgs


def combine_images(imgs):
    """Concatenate images horizontally"""
    widths, heights = zip(*(i.size for i in imgs))
    total_w = sum(widths)
    max_h = max(heights)
    out = Image.new("RGB", (total_w, max_h), (255, 255, 255))
    x_offset = 0
    for im in imgs:
        out.paste(im, (x_offset, 0))
        x_offset += im.size[0]
    return out


# =====================================
# 4. COMPLETENESS CHECK
# =====================================

def completeness_score(mesh):
    """
    Check if model is complete (no major holes)
    Based on: watertightness, boundary edges, Euler characteristic
    """
    scores = []
    
    # Watertight = complete
    if mesh.is_watertight:
        scores.append(1.0)
    else:
        # Check boundary ratio
        boundary_edges = mesh.edges_unique[mesh.edges_face.flatten() == -1]
        boundary_ratio = len(boundary_edges) / len(mesh.edges_unique)
        scores.append(1 - boundary_ratio)
    
    # Euler characteristic check (should be 2 for closed sphere-like objects)
    V = len(mesh.vertices)
    E = len(mesh.edges_unique)
    F = len(mesh.faces)
    euler = V - E + F
    
    # Deviation from ideal (2 for sphere)
    euler_score = 1.0 / (1.0 + abs(euler - 2) / 10.0)
    scores.append(euler_score)
    
    return float(np.mean(scores))


# =====================================
# MAIN EVALUATION
# =====================================

def evaluate_absolute_quality(obj_paths, text_prompt=None):
    """
    Evaluate single model quality without ground truth
    
    Returns quality scores in multiple dimensions:
    - Geometry: smoothness, triangle quality, resolution
    - Topology: manifold, watertight, completeness
    - Semantic: text-image alignment (if prompt provided)
    """
    
    print(f"\n{'='*60}")
    print("Absolute 3D Model Quality Evaluation")
    print(f"{'='*60}\n")
    
    # Load model
    print(f"Loading: {obj_path}")
    try:
        mesh = trimesh.load(obj_path, force='mesh')
        print(f"✓ Loaded: {len(mesh.vertices)} vertices, {len(mesh.faces)} faces\n")
    except Exception as e:
        print(f"❌ Failed to load: {e}")
        return None
    
    results = {}
    
    # 1. GEOMETRIC QUALITY
    print("▶ Geometric Quality")
    print("─" * 60)
    
    try:
        smoothness = surface_smoothness(mesh)
        results['smoothness'] = smoothness
        status = "✓" if smoothness < 0.3 else "⚠️" if smoothness < 0.5 else "❌"
        print(f"  {status} Surface Smoothness:     {smoothness:.4f} (< 0.3 is good)")
    except Exception as e:
        print(f"  ❌ Smoothness: {e}")
    
    try:
        tri_quality = triangle_quality(mesh)
        results['triangle_quality'] = tri_quality
        status = "✓" if tri_quality['bad_triangle_ratio'] < 0.1 else "⚠️" if tri_quality['bad_triangle_ratio'] < 0.3 else "❌"
        print(f"  {status} Triangle Aspect Ratio:  {tri_quality['mean']:.2f} ± {tri_quality['std']:.2f}")
        print(f"     Bad triangles (>5:1):    {tri_quality['bad_triangle_ratio']*100:.1f}%")
    except Exception as e:
        print(f"  ❌ Triangle quality: {e}")
    
    try:
        res = resolution_score(mesh)
        results['resolution'] = res
        print(f"  ℹ️  Vertex Density:          {res['vertices_per_unit_area']:.1f} verts/unit²")
        print(f"     Surface Area:            {res['surface_area']:.2f}")
    except Exception as e:
        print(f"  ❌ Resolution: {e}")
    
    # 2. TOPOLOGY QUALITY
    print("\n▶ Topology Quality")
    print("─" * 60)
    
    try:
        topo_score = topology_quality_score(mesh)
        results['topology_score'] = topo_score
        status = "✓" if topo_score > 0.8 else "⚠️" if topo_score > 0.5 else "❌"
        print(f"  {status} Overall Topology:      {topo_score:.4f} (0-1, higher better)")
        print(f"     Watertight:              {mesh.is_watertight}")
    except Exception as e:
        print(f"  ❌ Topology: {e}")
    
    try:
        completeness = completeness_score(mesh)
        results['completeness'] = completeness
        status = "✓" if completeness > 0.8 else "⚠️" if completeness > 0.5 else "❌"
        print(f"  {status} Completeness:           {completeness:.4f} (0-1, higher better)")
    except Exception as e:
        print(f"  ❌ Completeness: {e}")
    
    # 3. SEMANTIC COHERENCE
    if text_prompt:
        print("\n▶ Semantic Quality")
        print("─" * 60)
        try:
            semantic = semantic_coherence(mesh, text_prompt)
            results['semantic_coherence'] = semantic
            if semantic is not None:
                status = "✓" if semantic > 0.25 else "⚠️" if semantic > 0.15 else "❌"
                print(f"  {status} Text-Image Match:       {semantic:.4f} (prompt: '{text_prompt}')")
        except Exception as e:
            print(f"  ❌ Semantic: {e}")
    
    # OVERALL SCORE
    print(f"\n{'='*60}")
    print("Overall Quality Score")
    print(f"{'='*60}")
    
    # Weighted average of available scores
    score_weights = {
        'topology_score': 0.3,
        'completeness': 0.2,
        'smoothness': 0.2,  # Inverted: 1 - smoothness
        'triangle_quality': 0.2,
        'semantic_coherence': 0.1
    }
    
    weighted_scores = []
    for key, weight in score_weights.items():
        if key in results and results[key] is not None:
            if key == 'smoothness':
                # Invert: lower smoothness = better
                value = max(0, 1 - results[key] / 0.5)
            elif key == 'triangle_quality':
                # Use bad triangle ratio (inverted)
                value = 1 - results[key]['bad_triangle_ratio']
            else:
                value = results[key]
            
            weighted_scores.append(value * weight)
    
    if weighted_scores:
        overall = sum(weighted_scores) / sum(score_weights.values())
        results['overall_quality'] = overall
        
        if overall > 0.8:
            grade = "A (Excellent)"
            emoji = "🌟"
        elif overall > 0.6:
            grade = "B (Good)"
            emoji = "✓"
        elif overall > 0.4:
            grade = "C (Fair)"
            emoji = "⚠️"
        else:
            grade = "D (Poor)"
            emoji = "❌"
        
        print(f"  {emoji} Overall Quality: {overall:.4f} - Grade: {grade}")
    
    print(f"{'='*60}\n")
    
    return results


# =====================================
# BATCH COMPARISON (Rank multiple models)
# =====================================

def compare_multiple_models(obj_paths, text_prompt=None):
    """
    Evaluate and rank multiple models
    """
    print(f"\n{'='*60}")
    print(f"Comparing {len(obj_paths)} Models")
    print(f"{'='*60}\n")
    
    all_results = []
    
    for i, path in enumerate(obj_paths, 1):
        print(f"\n[Model {i}/{len(obj_paths)}]")
        results = evaluate_absolute_quality(path, text_prompt)
        if results and 'overall_quality' in results:
            all_results.append({
                'path': path,
                'score': results['overall_quality'],
                'details': results
            })
    
    # Rank by overall quality
    all_results.sort(key=lambda x: x['score'], reverse=True)
    
    print(f"\n{'='*60}")
    print("RANKING")
    print(f"{'='*60}")
    
    for i, r in enumerate(all_results, 1):
        print(f"  {i}. {r['path']}")
        print(f"     Score: {r['score']:.4f}")
    
    print(f"{'='*60}\n")
    
    return all_results


# =====================================
# ENTRY POINT
# =====================================

if __name__ == "__main__":
    import sys
    
    if len(sys.argv) < 2:
        print("Usage:")
        print("  Single model:   python absolute_quality_eval.py model.obj")
        print("  With prompt:    python absolute_quality_eval.py model.obj --prompt 'a hair dryer'")
        print("  Multiple:       python absolute_quality_eval.py model1.obj model2.obj --prompt 'a chair'")
        sys.exit(1)
    
    # Parse arguments
    obj_files = [arg for arg in sys.argv[1:] if arg.endswith('.obj')]
    text_prompt = None
    
    for i, arg in enumerate(sys.argv):
        if arg == '--prompt' and i + 1 < len(sys.argv):
            text_prompt = sys.argv[i + 1]
    
    if len(obj_files) == 1:
        evaluate_absolute_quality(obj_files[0], text_prompt)
    else:
        compare_multiple_models(obj_files, text_prompt)

SyntaxError: invalid decimal literal (3355891871.py, line 255)

In [2]:
python absolute_quality_eval.py model_tripo.obj --prompt "a hair dryer"
python absolute_quality_eval.py model_hunyuan.obj --prompt "a hair dryer"

SyntaxError: invalid syntax (3417023128.py, line 1)